# Wstęp

Celem projektu było przygotowanie i przetestowanie modelu klasyfikującego proste rysunki należące do dziesięciu różnych klas.

Wykorzystano framework **PyTorch** oraz architekturę **ResNet-18** z wagami wstępnie wytrenowanymi na dużym zbiorze obrazów.

W ramach pracy przeprowadzono analizę i wstępne przetwarzanie danych, podział zbioru na części treningową, walidacyjną i testową, a następnie proces uczenia i ewaluacji modelu.

In [ ]:
"""
IMPORTY
"""

from torch import optim
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score, f1_score)
from src.dataset import explore
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import splitfolders
import torch.nn as nn
import numpy as np
import itertools

In [ ]:
RANDOM_SEED = 67
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Eksploracja danych

W tym kroku wykorzystano funkcję `explore(DATA_ROOT, RANDOM_SEED)` z modułu `src.dataset`, aby **obejrzeć strukturę i przykładowe obrazy** oraz sprawdzić rozkład klas. Zbiór zawiera 10 klas szkiców/symboli:

`anchor, balloon, bicycle, envelope, paper_boat, peace_symbol, smiley, speech_bubble, spiral, thumb`.

**Po co eksploracja?** Pozwala szybko zweryfikować, czy dane są poprawnie zorganizowane w katalogach, czy klasy są zbalansowane oraz jak wyglądają „trudne” przypadki (np. podobne kształty). Na tej podstawie można lepiej zaplanować preprocessing i strategię uczenia (np. czy potrzebne będą augmentacje).

In [ ]:
DATA_ROOT = "../data"
explore(DATA_ROOT, RANDOM_SEED)

# Wnioski z eksploracji

Z analizy danych widać, że:
- Klasy są zbalansowane.
- Wszystkie obrazy mają taką samą rozdzielczość.

Rysunki wykonane ręcznie lub jako pieczątka mają trochę inne tło, często jest ciemniejsze albo widać na nim kratki czy linie.
Nie dotyczy to jednak tylko jednej klasy, więc nie powinno to negatywnie wpłynąć na uczenie modelu.


# Przygotowanie zbiorów

Dane podzielono automatycznie na trzy części przy pomocy `splitfolders.ratio`:
- **train: 70%**
- **val: 15%**
- **test: 15%**

Taki podział daje wystarczającą liczbę przykładów do nauki, jednocześnie pozostawiając osobny zbiór testowy do końcowej oceny. Wydzielenie walidacji (15%) umożliwia monitorowanie jakości w trakcie trenowania i dobór hiperparametrów bez „podglądania” testu.

In [ ]:
input_folder = DATA_ROOT
output_folder = "../data_split"

splitfolders.ratio(
    input_folder,
    output=output_folder,
    seed=42,
    ratio=(.7, .15, .15),
    group_prefix=None,
    move=False
)

# Preprocessing

Zastosowane przekształcenia (dla `train` i `val`):
- `transforms.ToTensor()` – konwersja obrazu do tensora PyTorch.
- `transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])` – normalizacja do zakresu około **[-1, 1]**.

**Uzasadnienie:**
- Normalizacja przyspiesza i stabilizuje uczenie sieci konwolucyjnych (warstwy widzą dane w podobnej skali).
- Zachowano **identyczny preprocessing** dla `train` i `val/test`, aby miara jakości była porównywalna.
- Nie użyto augmentacji w tej wersji (świadomie, jako **bazowej konfiguracji**), aby najpierw sprawdzić jak daleko dojdziemy bez dodatkowych modyfikacji danych. W razie potrzeby można dodać np. `RandomRotation`, `RandomHorizontalFlip`, lekkie zmiany jasności/kontrastu, co zwykle poprawia uogólnianie na rysunkach/szkicach.

In [ ]:
data_transforms = {
    "train": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    "val": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

train_dir = "../data_split/train"
valid_dir = "../data_split/val"
test_dir = "../data_split/test"

train_ds = datasets.ImageFolder(train_dir, data_transforms["train"])
valid_ds = datasets.ImageFolder(valid_dir, data_transforms["val"])
test_ds = datasets.ImageFolder(test_dir, data_transforms["val"])

print("Kontrolne sprawdzenie klas:", train_ds.classes)

BATCH = 4
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=4, pin_memory=True)
val_loader   = DataLoader(valid_ds,   batch_size=BATCH, shuffle=True, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=True, num_workers=4, pin_memory=True)

data_loaders = {"train": train_loader, "val": val_loader}
dataset_sizes = {"train": len(train_loader), "val": len(val_loader)}


images, labels = next(iter(train_loader))
images = images * 0.5 + 0.5

grid = torchvision.utils.make_grid(images[:16], nrow=8, padding=2)

plt.figure(figsize=(12, 4))
plt.imshow(grid[0], cmap="gray")
plt.axis("off")
plt.title("Podgląd obrazów po preprocessingu")
plt.show()



# Model

Wybrano **ResNet-18** z wagami wstępnie wytrenowanymi `weights='ResNet18_Weights.DEFAULT'` (transfer learning). Strategia uczenia:
- **Zamrożono** wszystkie warstwy poza ostatnią (`fc`) – uczymy jedynie klasyfikator.
- Funkcja kosztu: **CrossEntropyLoss**.
- Optymalizator: **SGD** z `lr=0.001` i `momentum=0.9`.
- Urządzenie: **CUDA** jeśli dostępna, w przeciwnym razie CPU.

**Dlaczego tak?**
- ResNet‑18 jest lekki i szybki, a wstępnie wytrenowane cechy konwolucyjne dobrze przenoszą się na nowe zadania wizji komputerowej nawet przy **niewielkiej liczbie danych**.
- Zamrożenie backbone’u ogranicza **przeuczenie** i znacząco skraca czas treningu.
- Użyto optymalizatora SGD z momentum, bo jest dość prosty i stabilny. Działa przewidywalnie i często daje lepsze wyniki niż metody takie jak Adam czy RMSProp, które szybciej się uczą, ale czasem prowadzą do gorszego uogólnienia modelu.

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = models.resnet18(weights='ResNet18_Weights.DEFAULT')

for name, param in model.named_parameters():
    if "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model = model.to(DEVICE)


In [ ]:
EPOCHS = 30

for epoch in range(EPOCHS):
    for phase in ["train", "val"]:
        if phase == "train":
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in data_loaders[phase]:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == "train"):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == "train":
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        num_samples = len(data_loaders[phase].dataset)
        epoch_acc = running_corrects.double().item() / num_samples

        epoch_loss = running_loss / dataset_sizes[phase]


        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

print("Training complete")

In [ ]:
torch.save(model.state_dict(), f"model.pth")

# Ewaluacja

Ewaluację przeprowadzono na **zbiorze testowym** (niewykorzystywanym podczas trenowania). Obliczono:
- **Accuracy** oraz **macro F1**,
- pełny **classification report** z precyzją i czułością dla każdej klasy,
- **macierz pomyłek** (Confusion Matrix) z wizualizacją.

**Wyniki (uzyskane na własnych testach):**
- **Accuracy: 0.9595**
- **Macro F1: 0.9592**
- Przykładowe klasy: `envelope`, `smiley`, `spiral` osiągnęły **F1 = 1.0**;   słabsze wyniki odnotowano m.in. dla `speech_bubble` (**F1 ≈ 0.86**) oraz `thumb` (**precision ≈ 0.80**, recall = 1.0) i `paper_boat` (recall ≈ 0.86).

**Interpretacja:**
- Wysokie makro‑F1 potwierdza, że model **dobrze rozróżnia** klasy.
- Pomyłki skupiają się na wizualnie podobnych kształtach co widać na macierzy pomyłek, mogą wynikać z braku augmentacji.

In [ ]:
model = models.resnet18(weights='ResNet18_Weights.DEFAULT')
model.fc = nn.Linear(model.fc.in_features, 1000)
model.load_state_dict(torch.load("model.pth", weights_only=True))
model.eval()

new_model = models.resnet18(weights='ResNet18_Weights.DEFAULT')
new_model.fc = nn.Linear(model.fc.in_features, 10)
new_model.fc.weight.data = model.fc.weight.data[0:2]
new_model.fc.bias.data = model.fc.bias.data[0:2]

In [ ]:
all_logits, all_probs, all_preds, all_targets, all_paths = [], [], [], [], []
classes = ['anchor', 'balloon', 'bicycle', 'envelope', 'paper_boat', 'peace_symbol', 'smiley', 'speech_bubble', 'spiral', 'thumb']
class_names = test_ds.classes
num_classes = len(classes)

with torch.no_grad():
    for imgs, y in test_loader:
        imgs = imgs.to(DEVICE)
        logits = model(imgs)
        probs = torch.softmax(logits, dim=1)[:,1]
        preds = logits.argmax(1).cpu()

        all_logits.append(logits.cpu())
        all_preds.append(preds)
        all_targets.append(y)
        all_probs.append(probs.cpu())


logits = torch.cat(all_logits).numpy()
preds  = torch.cat(all_preds).numpy()
targets= torch.cat(all_targets).numpy()
probs  = torch.cat(all_probs).numpy()
paths  = np.array(test_ds.samples)[:,0]


acc = accuracy_score(targets, preds)
f1m = f1_score(targets, preds, average="macro")
print(f"\nAccuracy: {acc:.4f} | Macro F1: {f1m:.4f}\n")
print(classification_report(targets, preds, target_names=class_names, digits=4))


cm = confusion_matrix(targets, preds, labels=list(range(num_classes)))
plt.figure(figsize=(6,5))
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion matrix")
plt.colorbar()
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, class_names, rotation=45, ha='right')
plt.yticks(tick_marks, class_names)
th = cm.max()/2
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center",
             color="white" if cm[i, j] > th else "black")
plt.ylabel('Prawdziwa klasa'); plt.xlabel('Predykcja')

# Wnioski

1. **Skuteczność:** Prosty baseline z ResNet‑18 + zamrożony backbone + uczenie tylko warstwy `fc` dał **~96% accuracy** i **~0.96 macro‑F1** na zbiorze testowym (74 obrazów).
2. **Co zadziałało:**
   - Transfer learning z gotowymi wagami.
   - Podział danych (70/15/15) i spójny preprocessing.
3. **Miejsca do poprawy:**
   - Dodać **augmentacje** (rotacje, odbicia, delikatne szumy/kontrast) to powinno ograniczyć pomyłki w podobnych klasach.
   - Zapisywać **najlepszy model** na walidacji (checkpoint) i ładować go do testów.
   - W kolejnych eksperymentach można rozważyć zmniejszenie liczby epok (np. do 20) przy zastosowaniu early stopping, co skróciłoby czas treningu bez pogorszenia wyników.
4. **Dlaczego wybrano takie rozwiązania:**
   - Celem było uzyskanie **solidnej, szybkiej bazy** bez skomplikowanych zależności i dlatego zamrożono backbone i użyto SGD.
   - Pozwala to łatwo rozszerzyć pipeline: włączyć augmentacje, odblokować część warstw, podstroić hiperparametry, gdy zajdzie potrzeba.

**Podsumowanie:** Otrzymany pipeline jest **czysty, powtarzalny i skuteczny**. Zaproponowane usprawnienia powinny pomóc przekroczyć 96% oraz poprawić najtrudniejsze klasy.